# Make a retarded amount of features
##  Importing Necessary Libraries:
 
* Importing essential libraries for data manipulation, visualization, and machine learning.

In [1]:
#Importing the necessary libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
input_data_path = '../data'

In [5]:
# Load the training data
train_logs = pd.read_csv(f'{input_data_path}/train_logs.csv')
train_scores = pd.read_csv(f'{input_data_path}/train_scores.csv')

## Merging Logs and Scores:

* Merging the two datasets based on the unique essay ID.

In [6]:
# Merge logs and scores based on essay ID
train_data = pd.merge(train_logs, train_scores, on='id')

## Check out the data

In [15]:
train_data.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count,score
count,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06
mean,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,1.222964e+03,2.314687e+02,3.992162e+00
std,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,9.485242e+02,1.759088e+02,9.918788e-01
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-01
25%,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,4.990000e+02,9.600000e+01,3.500000e+00
50%,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,1.043000e+03,2.000000e+02,4.000000e+00
75%,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,1.706000e+03,3.270000e+02,4.500000e+00
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03,6.000000e+00


In [38]:
train_data.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,3.5
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,3.5
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,3.5


In [27]:
train_data.loc[train_data['down_event'] != train_data['up_event']]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
5115,0042269b,105,104507,104617,110,Replace,q,m,qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq...,1,4,6.0
5232,0042269b,222,158578,158638,60,Replace,q,i,qqqqqqq => q,15,14,6.0
6276,0042269b,1266,473329,473439,110,Replace,q,o,qqq qq qqq qqqqqqq qqqqqq qqqq qq => q,194,128,6.0
6825,0042269b,1815,641821,641920,99,Replace,q,t,qqq qqqqq qq qqqqqqqq qqqq qqq qqqqqqq qq qqqq...,529,190,6.0
8223,0042269b,3213,1410198,1410293,95,Replace,q,a,qqqqqqq qqqqqqqqq qqqq qqq qq qqqq qqq qqqqqqq...,1539,282,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8323360,fde20dd8,2019,495093,495205,112,Replace,q,i,qqqq => q,530,167,4.0
8328686,fde20dd8,7345,1760415,1760572,157,Replace,q,y,qq qqq qqq'q qqqqq qqqqqq qq qqq qqqqqqq => q,3218,700,4.0
8378372,ff90a677,29,50477,50647,170,Nonproduction,a,A,NoChange,18,4,4.0
8378374,ff90a677,31,50703,50837,134,Replace,q,i,q qq qqqqqqq qqqq => q,1,1,4.0


## Thoughts  

We create a model where the events of an essay are encoded in some way and then are input into our model.  

How are the results for one essay encoded? A few ways:

* Aggregation 
    * There is a loss of information in our data when we create aggregations
* Features that take entire "time series" of a given essay and encode it to have closer to "lossless" feature creation
    * We'll look into this later

## Aggregation

### Text change

In [22]:
# Function for pandas apply that check number of large text changes
def count_large_text_changes(text_changes):
    return len([tc for tc in text_changes if len(tc) > 20])

def count_extremely_large_text_changes(text_changes):
    return len([tc for tc in text_changes if len(tc) > 100])

### Activity

In [35]:
unique_activities = train_data['activity'].unique()

# There are many many different types of Move From activities, so we'll remove them all and we'll treat them differently
non_move_activities = [a for a in unique_activities if 'Move From' not in a]

In [36]:
non_move_activities

['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste']

**Count of activities**

In [37]:
# For a given type of activity, count the number of times it occurs
def count_nonproduction(action_type, action_list):
    action_type = 'Nonproduction'
    return len([action for action in action_list if action == action_type])

# count input
def count_input(action_type, action_list):
    action_type = 'Input'
    return len([action for action in action_list if action == action_type])

# count remove/cut
def count_remove(action_type, action_list):
    action_type = 'Remove/Cut'
    return len([action for action in action_list if action == action_type])

# Count Replace
def count_replace(action_type, action_list):
    action_type = 'Replace'
    return len([action for action in action_list if action == action_type])

# Count Paste
def count_paste(action_type, action_list):
    action_type = 'Paste'
    return len([action for action in action_list if action == action_type])

**Move From features**

In [42]:
# Get all entries where the activity is a Move From
move_from_entries = train_data.loc[train_data['activity'].str.contains('Move From')]
move_from_entries.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
2359,001519c8,2360,1380334,1380334,0,"Move From [284, 292] To [282, 290]",Leftclick,Leftclick,qqqqqqq,290,244,3.5
2362,001519c8,2363,1382896,1382896,0,"Move From [287, 289] To [285, 287]",Leftclick,Leftclick,qq,287,244,3.5
2515,001519c8,2516,1735021,1735021,0,"Move From [460, 461] To [465, 466]",Leftclick,Leftclick,q,466,256,3.5
263545,07bb2245,2374,1282273,1282273,0,"Move From [905, 1314] To [907, 1316]",Leftclick,Leftclick,qqqqq qqqq qqqqqqq qqqq qqqq qqqqqq qqqqqqqq ...,1316,275,3.5
263572,07bb2245,2401,1650084,1650084,0,"Move From [565, 743] To [669, 847]",Leftclick,Leftclick,qqqq qqqqqq q qqqq qq qqqqq qqqq qqqq qqqqqqq....,847,275,3.5


In [46]:
import re

In [77]:
vectors = [np.array(a[1:-1].split(', '), dtype=int) for a in re.findall(r'\[[0-9]*, [0-9]*\]', move_from_entries['activity'].iloc[0])]

In [78]:
vectors

[array([284, 292]), array([282, 290])]

In [79]:
np.subtract(vectors[0], vectors[1])

array([2, 2])

## Add the move to feature to the agg function

## Features to create

* Actual word count
* Length of an action
* Last word count
* Amount of paragraphs (number of times /n is used)
* Create time series features and then aggregate them
* Length it took to write
* Did they review (reread) what they wrote?
* Distance of mouse movement 
* Angle of mouse movement 
* Movement around essay with non mouse movement 
* Different common patterns of activities (Maybe take a look at X sized window of actions and determine which ones are more commmon amongst different scoring groups)
* Essay structure reconstruction
    * What is the structure of the essay
    * Average size of 
* Usage of punctuation
    * Double quotations --> How many citations are there? 
    * Square brackets --> References
* Time allocation
    * What is the average time of their pauses?
    * What is the pattern of the pauses?
    * What is the patterns/length of pauses at the beginning of the essay writing?
    * What is the patterns/length of pauses at the end of the essay writing?
* Perform clustering on the features in order to try and seperate them into 4 groups (4 prompts)
* perplexity --> LLM scores
* Words per minute --> Speed of typing
* Introduction is for the end
    * Can you isolate when the paragraphs were written in the course of the total time of the essay writing)


In [41]:
# Calculate correlation of cursor position and word count
train_data['cursor_position'].corr(train_data['word_count'])

0.9113947240991126

In [ ]:
# Length of action

In [23]:
# Feature engineering for typing behavior features
typing_features = train_data.groupby('id').agg({
    'activity': 'count',                # Total number of activities
    'action_time': ['sum', 'mean'],     # Total and average action time
    'word_count': 'max',                # Maximum word count
    'text_change': 'nunique',           # Number of unique text changes
    'cursor_position': 'mean',           # Average cursor position
    'text_change' : count_large_text_changes,
    'text_change' : count_extremely_large_text_changes,
    'activity': count_nonproduction,
    'activity': count_input,
    'activity': count_remove,
    'activity': count_replace,
    'activity': count_paste,
    
})

In [24]:
typing_features.head()

activity action_time             word_count              text_change  \
            count         sum        mean        max count_large_text_changes   
id                                                                              
001519c8     2557      297243  116.246774        256                        1   
0022f953     2454      275391  112.221271        323                        0   
0042269b     4136      421201  101.837766        404                        8   
0059420b     1556      189596  121.848329        206                        0   
0075873a     2531      313702  123.943896        252                        0   

         cursor_position  
                    mean  
id                        
001519c8      711.163473  
0022f953      776.205786  
0042269b      731.611702  
0059420b      542.537275  
0075873a      600.050968

## Flatten the Multi-Level Column Index:

* Flattening the multi-level column index for ease of use.

In [ ]:
# Flatten the multi-level column index
typing_features.columns = ['_'.join(col).strip() for col in typing_features.columns.values]


## Merging Typing Features with Scores:

* Merging the typing behavior features with essay scores.

In [ ]:
# Merge typing features with scores
features = typing_features.merge(train_scores, on='id')